In [2]:
import pymysql
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='test_ocw',
                             charset='utf8',
                             # Selectの結果をdictionary形式で受け取る
                             cursorclass=pymysql.cursors.DictCursor)

In [8]:
import requests
from bs4 import BeautifulSoup

'''
OCWから学院一覧を取得するスクリプト(6個くらいだから必要ない気もする)
gakuinListの各要素は次のような辞書に鳴っている
{
	'name' : 学院名,
	'url' : その学院の授業の一覧のurl,
}
'''
def getGakuinList():
	url = "http://www.ocw.titech.ac.jp/"
	response = requests.get(url)
	soup = BeautifulSoup(response.content,"lxml")
    
	print("Get GakuinList")

	topMainNav = soup.find("ul",id="top-mein-navi")

	gakubus = topMainNav.find_all(class_="gakubuBox")

	gakuinList = []
	for gakubu_div in gakubus:
		gakuin = gakubu_div.find(class_="gakubuHead").span.string
		#if gakuin[-2::] != "学院":
		#	continue
		gakuin_url = url + gakubu_div.parent['href']
		gakuinList.append({'gakuin':gakuin,'gakuin_url':gakuin_url})

	return gakuinList

'''
[{'name': '理学院', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0100&GakubuCD=1&lang=JA'},
{'name': '工学院', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0100&GakubuCD=2&lang=JA'},
{'name': '物質理工学院', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0100&GakubuCD=3&lang=JA'},
{'name': '情報理工学院', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0100&GakubuCD=4&lang=JA'},
{'name': '生命理工学院', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0100&GakubuCD=5&lang=JA'},
{'name': '環境・社会理工学院', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0100&GakubuCD=6&lang=JA'},
{'name': '教養科目群', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0200&GakubuCD=7&GakkaCD=370000&tab=2&focus=100&lang=JA'},
{'name': '類科目', 'url': 'http://www.ocw.titech.ac.jp//index.php?module=General&action=T0100&GakubuCD=0&lang=JA'}]
'''

'''
学院名とurlを渡されたらその学院の授業一覧を持ってくる
'''
def getLectures(name,url):
    urlprefix = "http://www.ocw.titech.ac.jp"
    response = requests.get(url)
    soup = BeautifulSoup(response.content,'lxml')
    
    print("\tGet Lectures:",name)
    
    tables = soup.find_all('table',class_='ranking-list')
    
    LecList = []

    for t in tables:
        table = t.tbody
        for item in table.find_all('tr'):
            code = item.find('td',class_='code').string
            name = item.find('td',class_='course_title').a.string #講義名
            lecture_url = urlprefix + item.find('td',class_='course_title').a['href']
            teachers = [te.string for te in item.find('td',class_='lecturer').find_all('a')]
            #quater = item.find('td',class_='opening_department').a.string	#TODO ちゃんととれてない
            if not name or not code:	# 文字列が空の場合はスキップ
                continue
            if code:
                code = code.strip()
            if name:
                name = name.strip()
            #if quater:
            #    quater = quater.strip()
            #print(name)
            #print(teachers)
            #print(lecture_url)
            #print(quater)

            LecList.append({"code":code,"name":name,"lecture_url":lecture_url})

    return LecList

#print(getGakuinList())
#getLectures('情報理工学院','http://www.ocw.titech.ac.jp/index.php?module=General&action=T0100&GakubuCD=4&lang=JA')

#Q.曜日・時限が複数になった時もリスト構造にするか(今はリスト構造になっていない)
#各教員にaタグが入っている前提で以下の様になっている
#アクセスランキングは必要に感じなかったので空

#時折text内に凄いへんな文字が入る可能性がある(元のOCWの書かれ方のせい)
def fetch_OCW(Gakuin,Lecture):
    response = requests.get(Lecture["lecture_url"])
    soup = BeautifulSoup(response.content, "html.parser")
    print("\t\tSCRAPE Lecture:",Lecture["name"])
    
    OCW = {}

    for key in soup.find(attrs={"class":"gaiyo-data clearfix"}).find_all("dl"): #上部
        value_list = []
        for value in key.dd.strings:
            value = value.strip().replace(" ","").replace("\n","")
            if len(value)>0: value_list.append(value)
        OCW[key.dt.text] = ",".join(value_list)

    for key in soup.find_all(attrs={"class":"cont-sec"}): #下部
        keyString = key.h3.text

        if key.table is not None: #授業計画・学生が身につける力
            tr_list = []
            #print([j.text for j in key.thead.tr.find_all("th")]) #ヘッダ
            for TR in key.tbody.find_all("tr"): #ボディ
                tr_list.append("[{}]".format(
                    ",".join(["\\'{}\\'".format(j.text) for j in TR.find_all("td")])
                     ))
            OCW[keyString] = "[{}]".format(",".join(tr_list))
            ''' リスト構造で保持する場合
            for TR in i.tbody.find_all("tr"): #ボディ
                print([j.text.split() for j in TR.find_all("td")])
            '''
            
        elif key.ul is not None: #関連する科目
            OCW[keyString] = ",".join([j.text for j in key.ul.find_all("li")])
            
        else: #そのほか(文章で書かれた項目)
            OCW[keyString] = "\n".join(key.p.strings)
    
    #詳細ページに記載のなかった項目は適宜
    OCW["講義名"] = Lecture["name"]
    OCW["URL"] = Lecture["lecture_url"]
    if Gakuin["gakuin"][-2::] != "学院":
        OCW["学院"] = "その他"
    else:
        OCW["学院"] = Gakuin["gakuin"]
        
    return OCW

#データベースのカラム情報
#科目コードをキーとして持たせたい気持ち
column = {# TOP側 #
          "科目コード":"LectureCode", # キー
          "講義名":"LectureName",
          "担当教員名":"Professor",
          "開講元":"Department",
          "曜日・時限(講義室)":"DateRoom",
          "URL":"URL",
         #"講義室":"Room",
          "単位数":"Credit",
          "開講クォーター":"Quarter",
          "使用言語":"Language",
          # シラバス側 #
          "授業計画・課題":"LecturePlan",
          "成績評価の基準及び方法":"AssessStyle",
          "履修の条件(知識・技能・履修済科目等)":"CourseCond",
          # 検索用 #
          "学院":"Gakuin"}

#TABLEをつくる　データベースの構造が完全になったら不要
def createTable(column):
    with connection.cursor() as cursor:
        KEY_COLUMN = "科目コード"
        KEY_LENGTH = 10
        
        sub_column = []
        for k in column:
            if column[k] == KEY_COLUMN:
                #key処理そのいち
                sub_column.append(column[k]+" TEXT NOT NULL")
            else:
                sub_column.append(column[k]+" TEXT")
        
        sub_column.append("PRIMARY KEY({}({}))".format(column[KEY_COLUMN],KEY_LENGTH)) #key処理そのに
        sql = "CREATE TABLE lecture({});".format(",".join(sub_column))
        #print(sql)
        cursor.execute(sql)
        
#デバッグ用　TABLEを削除する
def dropTable():
    print("本当にOCW Tableを削除しますか？(する：y，しない：otherwise)")
    if input()=="y":
        with connection.cursor() as cursor:
            sql = "DROP TABLE lecture;"
            cursor.execute(sql)
    else:
        print("OCW Tableの削除を中止しました")
        
#講義情報をデータベースに格納する
def insertLecture(column,LectureData):
    with connection.cursor() as cursor:
        sql = "INSERT INTO lecture ({}) ".format(",".join(map(lambda x:column[x],column)))
        sql += "VALUES ({}) ".format(",".join(map(lambda x:"\'{}\'".format(LectureData[x]),column)))
        sql += "ON DUPLICATE KEY UPDATE {};".format(",".join(["{} = VALUES({})".format(column[k],column[k]) for k in list(filter(lambda x:x!="科目コード",column))]))
        cursor.execute(sql)
        print("\t\tUPDATE Lecture:",LectureData["講義名"])

In [18]:
#"科目コード":"LectureCode", # キー
#学院":"Gakuin"

def createLforGtable(column):
    with connection.cursor() as cursor:
        sub_column = []
        KEY_LENGTH = 20
        sub_column.append("{} NVARCHAR({}) NOT NULL".format(column["科目コード"],KEY_LENGTH))
        sub_column.append("{} NVARCHAR({}) NOT NULL".format(column["学院"],KEY_LENGTH))
        sub_column.append("PRIMARY KEY({},{})".format(column["科目コード"],column["学院"]))
        sql = "CREATE TABLE LforG({});".format(",".join(sub_column))
        #print(sql)
        cursor.execute(sql)
        
def dropLforGtable():
    print("本当にOCW LforG Tableを削除しますか？(する：y，しない：otherwise)")
    if input()=="y":
        with connection.cursor() as cursor:
            sql = "DROP TABLE LforG;"
            cursor.execute(sql)
    else:
        print("OCW Tableの削除を中止しました")
        
def insertLforG(column,code,gakuin):
    with connection.cursor() as cursor:
        sql = "INSERT IGNORE INTO LforG ({},{}) ".format(column["科目コード"],column["学院"])
        sql += "VALUES (\'{}\',\'{}\') ".format(code,gakuin)
        cursor.execute(sql)
        print("\t\tUPDATE LforG:",code,"-",gakuin)

In [9]:
#OCWスクレイピング実行
if __name__=='__main__':
    #limit値　越えて設定した場合，要素数ぶんが最大になる
    Glimit = 1 #頭からいくつ学院数見るか
    Llimit = 1 #頭からいくつ講義詳細見るか
    
    for Gakuin in getGakuinList()[:Glimit]:
        for Lecture in getLectures(Gakuin["gakuin"],Gakuin["gakuin_url"])[:Llimit]:
            OCWData = fetch_OCW(Gakuin,Lecture)
            insertLecture(column,OCWData)
            connection.commit()

Get GakuinList
	Get Lectures: 理学院
		SCRAPE Lecture: 磁気浮上と磁気支持工学
	UPDATE Lecture: 磁気浮上と磁気支持工学


In [19]:
dropLforGtable()
createLforGtable(column)

本当にOCW LforG Tableを削除しますか？(する：y，しない：otherwise)
a
OCW Tableの削除を中止しました


/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (3720, 'NATIONAL/NCHAR/NVARCHAR implies the character set UTF8MB3, which will be replaced by UTF8MB4 in a future release. Please consider using CHAR(x) CHARACTER SET UTF8MB4 in order to be unambiguous.')
  self._do_get_result()


In [24]:
for Gakuin in getGakuinList():
    for Lecture in getLectures(Gakuin["gakuin"],Gakuin["gakuin_url"]):
        insertLforG(column,Lecture["code"],Gakuin["gakuin"])

Get GakuinList
	Get Lectures: 理学院
		UPDATE LforG: EEE.P501 - 理学院
		UPDATE LforG: PHY.Q206 - 理学院
		UPDATE LforG: MEC.J431 - 理学院
		UPDATE LforG: MEC.E432 - 理学院
		UPDATE LforG: MAT.C404 - 理学院
		UPDATE LforG: MAT.M402 - 理学院
		UPDATE LforG: MTH.B211 - 理学院
		UPDATE LforG: MTH.D403 - 理学院
		UPDATE LforG: EEE.D401 - 理学院
		UPDATE LforG: EEE.D501 - 理学院
		UPDATE LforG: ENR.J409 - 理学院
		UPDATE LforG: PHY.M204 - 理学院
		UPDATE LforG: EEE.P411 - 理学院
		UPDATE LforG: MEC.G431 - 理学院
		UPDATE LforG: EEE.S451 - 理学院
		UPDATE LforG: PHY.Q207 - 理学院
		UPDATE LforG: CAP.E401 - 理学院
		UPDATE LforG: PHY.E212 - 理学院
		UPDATE LforG: PHY.Q311 - 理学院
		UPDATE LforG: MAT.P404 - 理学院
		UPDATE LforG: MAT.P422 - 理学院
		UPDATE LforG: CAP.I416 - 理学院
		UPDATE LforG: MAT.P491 - 理学院
		UPDATE LforG: ENR.A405 - 理学院
		UPDATE LforG: PHY.S312 - 理学院
		UPDATE LforG: MAT.C501 - 理学院
		UPDATE LforG: MAT.M406 - 理学院
		UPDATE LforG: CAP.A465 - 理学院
		UPDATE LforG: ENR.A408 - 理学院
		UPDATE LforG: ENR.K430 - 理学院
		UPDATE LforG: ENR.L410 - 理学院
		UPD

/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'MAT.M402-理学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'ENR.A407-理学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'CAP.T531-理学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'MAT.M407-理学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'CAP.T532-理学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'ENR.A403-理学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning

 理学院
		UPDATE LforG: MAT.M426 - 理学院
		UPDATE LforG: CAP.A424 - 理学院
		UPDATE LforG: CAP.A521 - 理学院
		UPDATE LforG: CAP.C441 - 理学院
		UPDATE LforG: CAP.I403 - 理学院
		UPDATE LforG: CAP.P421 - 理学院
		UPDATE LforG: MTH.B402 - 理学院
		UPDATE LforG: EPS.A451 - 理学院
		UPDATE LforG: ENR.A401 - 理学院
		UPDATE LforG: ENR.H405 - 理学院
		UPDATE LforG: PHY.F436 - 理学院
		UPDATE LforG: PHY.Q433 - 理学院
		UPDATE LforG: PHY.P622 - 理学院
		UPDATE LforG: CHM.D401 - 理学院
		UPDATE LforG: MEC.G531 - 理学院
		UPDATE LforG: EEE.D561 - 理学院
		UPDATE LforG: EEE.D551 - 理学院
		UPDATE LforG: MTH.A212 - 理学院
		UPDATE LforG: MTH.E654 - 理学院
		UPDATE LforG: ENR.A404 - 理学院
		UPDATE LforG: MTH.K501 - 理学院
		UPDATE LforG: ENR.B503 - 理学院
		UPDATE LforG: PHY.P648 - 理学院
		UPDATE LforG: CHM.C205 - 理学院
		UPDATE LforG: CHM.C204 - 理学院
		UPDATE LforG: CHM.D334 - 理学院
		UPDATE LforG: CHM.D305 - 理学院
		UPDATE LforG: EPS.B202 - 理学院
		UPDATE LforG: EPS.B330 - 理学院
		UPDATE LforG: EPS.Z381 - 理学院
		UPDATE LforG: CAP.A401 - 理学院
		UPDATE LforG: CAP.E521 - 理学院
		U

/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'ENR.A404-理学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'PHY.P616-理学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'ENR.A406-理学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'ENR.B503-理学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'CAP.E411-理学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'ENR.A402-理学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning

		UPDATE LforG: EPS.Z692 - 理学院
		UPDATE LforG: EPS.C531 - 理学院
		UPDATE LforG: ENR.E611 - 理学院
		UPDATE LforG: ENR.E601 - 理学院
		UPDATE LforG: ENR.E603 - 理学院
		UPDATE LforG: ENR.E609 - 理学院
		UPDATE LforG: ENR.E609 - 理学院
		UPDATE LforG: ENR.B502 - 理学院
		UPDATE LforG: PHY.C450 - 理学院
		UPDATE LforG: PHY.P414 - 理学院
		UPDATE LforG: PHY.P614 - 理学院
		UPDATE LforG: PHY.P615 - 理学院
		UPDATE LforG: PHY.P618 - 理学院
		UPDATE LforG: PHY.P619 - 理学院
		UPDATE LforG: PHY.P625 - 理学院
		UPDATE LforG: PHY.P627 - 理学院
		UPDATE LforG: PHY.Z696 - 理学院
		UPDATE LforG: PHY.P503 - 理学院
		UPDATE LforG: CHM.A441 - 理学院
		UPDATE LforG: CHM.A442 - 理学院
		UPDATE LforG: CHM.L472 - 理学院
		UPDATE LforG: CHM.A642 - 理学院
		UPDATE LforG: CHM.L572 - 理学院
		UPDATE LforG: MTH.B204 - 理学院
		UPDATE LforG: MTH.B212 - 理学院
		UPDATE LforG: MTH.B301 - 理学院
		UPDATE LforG: MTH.C331 - 理学院
		UPDATE LforG: EPS.C330 - 理学院
		UPDATE LforG: EPS.C660 - 理学院
		UPDATE LforG: EPS.C645 - 理学院
		UPDATE LforG: XIP.A601 - 理学院
		UPDATE LforG: MTH.K609 - 理学院
		UPDATE

...

		UPDATE LforG: UDE.Z491 - 環境・社会理工学院
		UPDATE LforG: UDE.Z683 - 環境・社会理工学院
		UPDATE LforG: UDE.Z692 - 環境・社会理工学院
		UPDATE LforG: UDE.Z593 - 環境・社会理工学院
		UPDATE LforG: UDE.S532 - 環境・社会理工学院
		UPDATE LforG: UDE.Z591 - 環境・社会理工学院
		UPDATE LforG: UDE.Z592 - 環境・社会理工学院
		UPDATE LforG: UDE.Y683 - 環境・社会理工学院
		UPDATE LforG: UDE.Y473 - 環境・社会理工学院
		UPDATE LforG: UDE.Y681 - 環境・社会理工学院
		UPDATE LforG: UDE.Y471 - 環境・社会理工学院
		UPDATE LforG: UDE.Y682 - 環境・社会理工学院
		UPDATE LforG: SHS.D441 - 環境・社会理工学院
		UPDATE LforG: SHS.S444 - 環境・社会理工学院
		UPDATE LforG: SHS.P462 - 環境・社会理工学院
		UPDATE LforG: SHS.D461 - 環境・社会理工学院
		UPDATE LforG: SHS.S462 - 環境・社会理工学院
		UPDATE LforG: SHS.L411 - 環境・社会理工学院
		UPDATE LforG: SHS.Z491 - 環境・社会理工学院
		UPDATE LforG: SHS.P681 - 環境・社会理工学院
		UPDATE LforG: SHS.P682 - 環境・社会理工学院
		UPDATE LforG: SHS.U681 - 環境・社会理工学院
		UPDATE LforG: SHS.S682 - 環境・社会理工学院
		UPDATE LforG: SHS.L671 - 環境・社会理工学院
		UPDATE LforG: SHS.L631 - 環境・社会理工学院
		UPDATE LforG: SHS.L632 - 環境・社会理工学院
		UPDATE LforG: SHS.Z696 - 環境・社会理工学院
	

/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'CVE.N333-環境・社会理工学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'CVE.Z381-環境・社会理工学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'ISC.A610-環境・社会理工学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'ISC.B610-環境・社会理工学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'ISC.B611-環境・社会理工学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'ISC.D610-環境・社会理工学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-pack

		UPDATE LforG: CVE.Z381 - 環境・社会理工学院
		UPDATE LforG: CAP.A464 - 環境・社会理工学院
		UPDATE LforG: CVE.N432 - 環境・社会理工学院
		UPDATE LforG: CVE.N433 - 環境・社会理工学院
		UPDATE LforG: CVE.P431 - 環境・社会理工学院
		UPDATE LforG: CVE.L401 - 環境・社会理工学院
		UPDATE LforG: CVE.P461 - 環境・社会理工学院
		UPDATE LforG: CVE.P612 - 環境・社会理工学院
		UPDATE LforG: CVE.N635 - 環境・社会理工学院
		UPDATE LforG: CVE.P601 - 環境・社会理工学院
		UPDATE LforG: CVE.Z692 - 環境・社会理工学院
		UPDATE LforG: CVE.Z693 - 環境・社会理工学院
		UPDATE LforG: CVE.Z591 - 環境・社会理工学院
		UPDATE LforG: CVE.P622 - 環境・社会理工学院
		UPDATE LforG: UDE.Z685 - 環境・社会理工学院
		UPDATE LforG: UDE.Z686 - 環境・社会理工学院
		UPDATE LforG: UDE.Z687 - 環境・社会理工学院
		UPDATE LforG: UDE.E501 - 環境・社会理工学院
		UPDATE LforG: UDE.Y593 - 環境・社会理工学院
		UPDATE LforG: SHS.U441 - 環境・社会理工学院
		UPDATE LforG: SHS.M443 - 環境・社会理工学院
		UPDATE LforG: SHS.M445 - 環境・社会理工学院
		UPDATE LforG: SHS.M446 - 環境・社会理工学院
		UPDATE LforG: SHS.L412 - 環境・社会理工学院
		UPDATE LforG: SHS.Z492 - 環境・社会理工学院
		UPDATE LforG: SHS.S681 - 環境・社会理工学院
		UPDATE LforG: SHS.M681 - 環境・社会理工学院
	

/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'ENR.R602-環境・社会理工学院' for key 'PRIMARY'")
  self._do_get_result()
/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1062, "Duplicate entry 'ENR.R603-環境・社会理工学院' for key 'PRIMARY'")
  self._do_get_result()


KeyboardInterrupt: 

In [ ]:
http://www.ocw.titech.ac.jp/index.php?module=General&action=T0300&GakubuCD=2&GakkaCD=321800&KeiCD=18&KougiCD=201807330&Nendo=2018&lang=JA&vid=03

In [ ]:
OCWData = fetch_OCW(Gakuin,Lecture)

In [66]:
dropTable()
createTable(column)

本当にOCW Tableを削除しますか？(する：y，しない：otherwise)
y


In [79]:
insertLecture(column,OCWData)

	UPDATE Lecture: 解析力学


In [16]:
#検索テスト
with connection.cursor() as cursor:
    sql = "SELECT * FROM lecture"
    cursor.execute(sql)
 
    dbdata = cursor.fetchall()
    for rows in dbdata:
        print(rows)

{'LectureCode': 'EEE.M211', 'LectureName': 'フーリエ変換とラプラス変換', 'Professor': '角嶋邦之,伊藤浩之', 'Department': '電気電子系', 'DateRoom': '水1-2(S221)\xa0\xa0金3-4(S221)', 'URL': 'http://www.ocw.titech.ac.jp/index.php?module=General&action=T0300&GakubuCD=2&GakkaCD=321700&KeiCD=17&KougiCD=201801976&Nendo=2018&lang=JA&vid=03', 'Credit': '2', 'Quarter': '2Q', 'Language': '日本語', 'LecturePlan': "[[\\'第1回\\',\\'線形システムとは，周期関数の級数展開\\',\\'線形システムの定義確認，フーリエ展開係数の導出\\'],[\\'第2回\\',\\'フーリエ級数の性質\\',\\'関数の偶奇性とフーリエ展開係数導出，フーリエ級数の項別微分\\'],[\\'第3回\\',\\'線形回路の周期波に対する定常応答\r\n\\',\\'回路の線形性判定，線形回路の定常応答と任意周期波の回路応答導出\\'],[\\'第4回\\',\\'フーリエ級数の理解度確認総合演習と解説\\',\\'第1回～第3回のポイント復習，演習と解説\\'],[\\'第5回\\',\\'非周期関数とフーリエ積分・フーリエ変換\\',\\'フーリエ変換・逆変換の式の導出，フーリエ余弦変換・フーリエ正弦変換\\'],[\\'第6回\\',\\'フーリエ変換の存在と性質\\',\\'フーリエ変換が存在するための条件の理解，フーリエ変換の性質\\'],[\\'第7回\\',\\'時間領域表示と周波数領域表示，線形回路の時間応答と周波数応答\\',\\'時間領域と周波数領域の間に存在する双対関係の説明，線形回路の時間応答と周波数応答の関係図示と説明\\'],[\\'第8回\\',\\'理解度の確認と解説\\',\\'第1回から第7回までの理解度確認と到達度自己評価\\'],[\\'第9回\\',\\'シャノンの標本化定理\\',\\'シャノンの標本化定理の説

In [3]:
#検索テスト
with connection.cursor() as cursor:
    sql = "SELECT * FROM LforG"
    cursor.execute(sql)
 
    dbdata = cursor.fetchall()
    for rows in dbdata:
        print(rows)

{'LectureCode': 'ARC.A202', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A203', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A204', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A205', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A301', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A302', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A304', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A305', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A307', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A401', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A402', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A404', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A405', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A406', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A407', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A408', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A505', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A506', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A507', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'ARC.A602', 'Ga

{'LectureCode': 'IEE.A431', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.A432', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.A433', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B201', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B202', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B203', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B204', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B205', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B206', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B207', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B208', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B301', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B302', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B330', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B331', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B332', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B333', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B334', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B335', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B336', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B337', 'Gakuin': '工学院'}
{'LectureCode': 'IEE.B401', 'Gakuin': '工学院'}
{'LectureC

{'LectureCode': 'MEC.H231', 'Gakuin': '工学院'}
{'LectureCode': 'MEC.H231', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'MEC.H331', 'Gakuin': '工学院'}
{'LectureCode': 'MEC.H431', 'Gakuin': '工学院'}
{'LectureCode': 'MEC.H431', 'Gakuin': '物質理工学院'}
{'LectureCode': 'MEC.H431', 'Gakuin': '理学院'}
{'LectureCode': 'MEC.H431', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'MEC.H432', 'Gakuin': '工学院'}
{'LectureCode': 'MEC.H433', 'Gakuin': '工学院'}
{'LectureCode': 'MEC.H433', 'Gakuin': '物質理工学院'}
{'LectureCode': 'MEC.H433', 'Gakuin': '理学院'}
{'LectureCode': 'MEC.H433', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'MEC.H434', 'Gakuin': '工学院'}
{'LectureCode': 'MEC.H434', 'Gakuin': '物質理工学院'}
{'LectureCode': 'MEC.H434', 'Gakuin': '理学院'}
{'LectureCode': 'MEC.H434', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 'MEC.H435', 'Gakuin': '工学院'}
{'LectureCode': 'MEC.H531', 'Gakuin': '工学院'}
{'LectureCode': 'MEC.H532', 'Gakuin': '工学院'}
{'LectureCode': 'MEC.I211', 'Gakuin': '工学院'}
{'LectureCode': 'MEC.I211', 'Gakuin': '環境・社会理工学院'}
{'LectureCode': 